 # Beveridge Curve

 The Beveridge curve, or UV curve, is a graphical representation of the relationship between unemployment and the job vacancy rate, the number of unfilled jobs expressed as a proportion of the labor force.

In [1]:
import altair as alt
import datapane as dp
import pandas as pd
import pandas_datareader as pdr

In [2]:
df = pdr.get_data_fred(["UNRATE", "JTSJOR"], start="2000-01-01")

In [3]:
df

,UNRATE,JTSJOR
DATE,,
2000-01-01,4.0,NaN
2000-02-01,4.1,NaN
2000-03-01,4.0,NaN
2000-04-01,3.8,NaN
2000-05-01,4.0,NaN
...,...,...
2022-11-01,3.6,6.5
2022-12-01,3.5,6.8
2023-01-01,3.4,6.4


In [4]:
def beveridge_factor(DF: pd.DataFrame) -> pd.DataFrame:
    return (
        DF.rename(
            columns={"UNRATE": "Unemployment Rate", "JTSJOR": "Job Openings Rate"}
        )
        .rename_axis(index={"DATE": "Date"})
        .assign(
            **{
                "Beveridge Factor": lambda d: d["Job Openings Rate"]
                / d["Unemployment Rate"]
            }
        )
    )

In [5]:
bv = df.pipe(beveridge_factor)

In [6]:
select_year = alt.selection_interval(encodings=["x"])

bar_slider = (
    alt.Chart(bv.reset_index())
    .mark_bar()
    .encode(x="year(Date)", y="mean(Beveridge Factor)")
    .properties(height=50, width=800)
    .add_selection(select_year)
)

plot = (
    alt.Chart(bv.reset_index())
    .mark_point(size=100)
    .encode(
        x="Unemployment Rate",
        y="Job Openings Rate",
        color="Job Openings Rate",
        tooltip=["Date", "Unemployment Rate", "Job Openings Rate"],
        opacity=alt.condition(select_year, alt.value(0.7), alt.value(0.1)),
    )
    .interactive()
).properties(
    title="The Beveridge Curve (job openings rate vs. unemployment rate), seasonally adjusted",
    width=800,
    height=400,
)
plot & bar_slider

alt.VConcatChart(...)

In [7]:
current_year = pd.Timestamp.today().year

In [8]:
view = dp.View(
    dp.Blocks(
        "# Beveridge Curve",
        dp.Formula(
            r"\text{Beveridge Factor} = \frac{\text{Job Openings Rate}}{\text{Unemployment Rate}}"
        ),
        dp.Plot(plot & bar_slider),
        "## Current Year",
        dp.Table(bv.loc[f"{current_year}"]),
    )
)

In [9]:
view

In [10]:
dp.upload_report(view, name="Beveridge Curve")

Uploading report and associated data - *please wait...*

Report successfully uploaded - view and share at <a href='https://cloud.datapane.com/reports/OkpKyP7/beveridge-curve/' target='_blank'>here</a>.

Uploaded CloudReport - view at https://cloud.datapane.com/reports/OkpKyP7/beveridge-curve/